# Lab_5
#### 1. В соответствии с вариантом задания сгенерировать экспериментальные данные, в которых в явном виде присутствует эффект мультиколлинеарности.
#### 2. Рассчитать ряд показателей, характеризующих эффект мультиколлинеарности. Определить факторы, ответственные за возникновение эффекта мультиколлинеарности.
#### 3. Построить ридж-оценки параметров при различных значениях параметра регуляризации. Выбрать оптимальное значение параметра регуляризации. Построить графики изменения квадрата евклидовой нормы оценок параметров и остаточной суммы квадратов от параметра регуляризации.
#### 4. Провести оценивание модели регрессии по методу главных компонентов. Перейти к описанию в исходном пространстве факторов. Сравнить решение с ридж-оцениванием по смещению оценок и точности предсказания отклика. 

## Инициализация программы

In [1]:
import numpy as np
import math


# Генерирует точки (120) в [-2.5; 2.5]
def generateData():
    import random as rnd

    x1_points = []
    x2_points = []
    x3_points = []
    x4_points = []
    x5_points = []
    
    for s in range(0, 120):
        x1_points.append((rnd.random() - 0.5) * 5)
        x2_points.append((rnd.random() - 0.5) * 5)
        x3_points.append((rnd.random() - 0.5) * 5)
        x4_points.append((rnd.random() - 0.5) * 5)
        x5_points.append((rnd.random() - 0.5) * 5)

    return x1_points, x2_points, x3_points, x4_points, x5_points

def getX(teta, f, x1, x2, x3, x4, x5):
    X = []
    for i in range(len(teta)):
        X.append([])
        for j in range(len(x1)):
            X[i].append(f(x1[j], x2[j], x3[j], x4[j], x5[j])[i])
    X = np.matrix(X)
    return X

def getY(u, x1, x2, x3, x4, x5, p = 0.1):
    u_arr = []
    for i in range(len(x1)):
        u_arr.append(u(x1, x2, x3, x4, x5))
        
    vect_arr_u = np.array(u_arr)
    average_u = vect_arr_u.mean()

    w2 = np.dot(vect_arr_u - average_u, vect_arr_u - average_u) / (len(vect_arr_u - 1))  
    sig = p * (w2)**0.5
    
    # Сгенерировали U с помехами (y)
    y = []
    for i in range(len(x1)):
        y.append(u_arr[i] + np.random.normal(0, sig))
        
    return y

def multVect(x, scalar):
    l = []
    for i in range(len(x)):
        l.append(x[i] * scalar)
    l = np.array(l)
    return l

## 1. В соответствии с вариантом задания сгенерировать экспериментальные данные, в которых в явном виде присутствует эффект мультиколлинеарности.

#### Регрессия на 7 факторах. Эффект мультиколлинеарности создают две пары факторов. Разброса в масштабах факторов нет;

In [2]:
# Эффект мультиколлинеарности создают две пары факторов
f = lambda x1, x2, x3, x4, x5: np.array([x1, x2, x3, x4, x5, x1 + 0.5 * x2, x1 * 0.5 + x2]) 
# Нет разброса в масштабах
teta = np.array([1, 1, 1, 1, 1, 1, 1]) 
def u(x1, x2, x3, x4, x5): 
    x1 = np.array(x1)
    x2 = np.array(x2)
    x3 = np.array(x3)
    x1 = np.array(x4)
    x1 = np.array(x5)
    x1_ = multVect(x1, 0.5)
    x2_ = multVect(x2, 0.5)
    return x1 + x2 + x3 + x4 + x5 + x1 + x2_ + x1_ + x2

x1, x2, x3, x4, x5 = generateData()

## 2. Рассчитать ряд показателей, характеризующих эффект мультиколлинеарности. Определить факторы, ответственные за возникновение эффекта мультиколлинеарности.

#### 1) Определитель информационной матрицы.

In [3]:
X = getX(teta, f, x1, x2, x3, x4, x5)
valDet = np.linalg.det((X.T).dot(X))
print('Определитель матрицы =', valDet)

Определитель матрицы = -0.0


#### 2) Минимальное собственное число матрицы XtX.

In [4]:
eigvalues, eigvectors = np.linalg.eig((X.T).dot(X))
minEig = min(eigvalues)
print('Минимальное собственное число =', minEig)

Минимальное собственное число = (-6.48959737708795e-14+0j)


#### 3) Мера обусловленности матрицы по Нейману-Голдстейну.

In [5]:
maxEig = max(eigvalues)
print('Мера обусловленности =', maxEig/minEig)

Мера обусловленности = (-1.3358807979879088e+16-0j)


#### 4) Максимальная парная сопряженность.

In [6]:
def sumF(x1, x2):
    l = []
    for i in range(len(x1)):
        l.append(x1[i] + x2[i])
    l = np.array(l)
    return l


       
x6 = sumF(x1, multVect(x2, 0.5))
x7 = sumF(multVect(x1, 0.5), x2)
matrix = np.array([x1, x2, x3, x4, x5, x6, x7])

R = []
for i in range(len(matrix)):
    R.append([])
    for j in range(len(matrix)):
        z = np.array(matrix[i]).T.dot(matrix[j])
        z /= np.linalg.norm(matrix[i]) * np.linalg.norm(matrix[j])
        R[i].append(z)
     
    
maxValue = -12123132 
index_i = 0 
index_j = 0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            if maxValue < R[i][j]:
                maxValue = R[i][j]
                index_i = i
                index_j = j
                
print(maxValue, index_i, index_j)

0.9019644786262714 1 6


#### 5) Максимальная сопряженность.

In [7]:
# Фиксируем независимую x3
P_sopr = np.matrix(R).conjugate() # Сопряженная
p_1 = np.linalg.inv(P_sopr) # Обратная
diag = np.diagonal(p_1)
Ri_2 = 1 - 1/diag
Ri_abs = abs(Ri_2**(1/2))
print(max(Ri_abs))

1.0


## 3. Построить ридж-оценки параметров при различных значениях параметра регуляризации. Выбрать оптимальное значение параметра регуляризации. Построить графики изменения квадрата евклидовой нормы оценок параметров и остаточной суммы квадратов от параметра регуляризации.

Для нахождения оптимального параметра регуляризации необходимо построить графики изменения квадрата евклидовой нормы оценок параметров и остаточной суммы квадратов от параметра регуляризации. Параметр 𝝀 будет определятся как некоторый компромисс между неизбежным увеличением остаточной  суммы квадратов и желаемым уменьшением евклидовой нормы оценок параметров.

In [8]:
y = getY(u, x1, x2, x3, x4, x5)
# LambdMAtr = np.diag(np.diagonal((X.dot(X.T)).dot(lamb)))
Teta_ = lambda lamb: (np.linalg.inv((X).dot(X.T) + np.diag(np.diagonal((X.dot(X.T)).dot(lamb)))).dot(X)).dot(y)
print(Teta_(1))

<ipython-input-1-35e536310435>:42: RuntimeWarning: invalid value encountered in sqrt
  sig = p * (w2)**0.5


ValueError: shape too large to be a matrix.